In [ ]:
from pydantic import Field, BaseModel, ConfigDict


class Report(BaseModel):
    """
    A report model that can be used to generate reports based on the session data.
    """

    model_config = ConfigDict(extra="forbid")

    title: str = Field(..., description="The title of the report")
    content: str = Field(..., description="The content of the report")


class ReportPackage(BaseModel):
    """
    A package of reports that can be used to generate a summary report.
    """

    good: Report = Field(..., description="A report that is considered good")
    bad: Report = Field(..., description="A report that is considered bad")


class ComplexityAssessment(BaseModel):
    """
    A model that assesses the complexity of a given task.
    """

    model_config = ConfigDict(extra="forbid")

    complexity_score: float = Field(
        ..., description="A score representing the complexity of the task"
    )
    details: str = Field(
        ..., description="Details about the complexity assessment"
    )


from openai import OpenAI

client = OpenAI()

responses = []
for i in range(10):
    response = client.responses.parse(
        model="gpt-4.1-nano",
        input="Given topic of journey to the west, generate two different story titles and a brief summary for each.",
        text_format=ComplexityAssessment,
    )
    responses.append(response)

complexity_scores = sum([i.complexity_score for i in responses])


def critic1_score(*args, **kwargs):
    return 0.5


def critic2_score(*args, **kwargs):
    """
    A critic operation that generates a report based on the session data.
    """
    return 0.3


text = """
<let lvar field_title option_one>money king on a jounrey<lvar/>
<let lvar field_title option_two>four monks and their horse<lvar/>

a = DO ACTION critic1_score(option_one) + DO ACTION critic2_score(option_one)
b = DO ACTION critic2_score(option_two) + DO ACTION critic2_score(option_two)

if *a > *b:
    return option_one
else:
    return option_two
"""


a = critic1_score(response.good.title) + critic2_score(response.good.title)

In [2]:
from lexer import Token, TokenType, Lexer
from parser import Parser

a = Lexer(text)
a.tokenize()

parser = Parser(a.tokens)

In [3]:
program = parser.parse()

In [4]:
program

Program(stmts=[CogVar(field_type='title', name='option_one', content='money king on a jounrey'), CogVar(field_type='title', name='option_two', content='four monks and their horse'), Let(name='a', expr=BinaryOp(left=FuncCall(name='critic1_score', args=[Identifier(name='option_one')]), operator='+', right=FuncCall(name='critic2_score', args=[Identifier(name='option_one')]))), Let(name='b', expr=BinaryOp(left=FuncCall(name='critic2_score', args=[Identifier(name='option_two')]), operator='+', right=FuncCall(name='critic2_score', args=[Identifier(name='option_two')]))), If(condition=BinaryOp(left=Identifier(name='*a'), operator='>', right=Identifier(name='*b')), then_body=[ExprStmt(expr=Identifier(name='return')), ExprStmt(expr=Identifier(name='option_one'))], else_body=[ExprStmt(expr=Identifier(name='return')), ExprStmt(expr=Identifier(name='option_two'))])])

In [6]:
program.stmts[0]

CogVar(field_type='title', name='option_one', content='money king on a jounrey')

In [7]:
for i in program.stmts:
    print(i)

CogVar(field_type='title', name='option_one', content='money king on a jounrey')
CogVar(field_type='title', name='option_two', content='four monks and their horse')
Let(name='a', expr=BinaryOp(left=FuncCall(name='critic1_score', args=[Identifier(name='option_one')]), operator='+', right=FuncCall(name='critic2_score', args=[Identifier(name='option_one')])))
Let(name='b', expr=BinaryOp(left=FuncCall(name='critic2_score', args=[Identifier(name='option_two')]), operator='+', right=FuncCall(name='critic2_score', args=[Identifier(name='option_two')])))
If(condition=BinaryOp(left=Identifier(name='*a'), operator='>', right=Identifier(name='*b')), then_body=[ExprStmt(expr=Identifier(name='return')), ExprStmt(expr=Identifier(name='option_one'))], else_body=[ExprStmt(expr=Identifier(name='return')), ExprStmt(expr=Identifier(name='option_two'))])


In [ ]:
# Test the fixed parser
from lexer import Token, TokenType, Lexer
from parser import Parser

text = """
<let lvar field_title option_one>money king on a jounrey<lvar/>
<let lvar field_title option_two>four monks and their horse<lvar/>

```lndl
a = DO ACTION critic1_score(option_one) + DO ACTION critic2_score(option_one)
b = DO ACTION critic2_score(option_two) + DO ACTION critic2_score(option_two)

if *a > *b:
    return option_one
else:
    return option_two
```
"""

# Test lexer
lexer = Lexer(text)
tokens = lexer.tokenize()

print("=== TOKENS ===")
for i, token in enumerate(tokens[:20]):  # Show first 20 tokens
    print(
        f"{i:2d}: {token.type.name:12} | {repr(token.value):20} | L{token.line}:C{token.column}"
    )

print(f"\n... showing first 20 of {len(tokens)} total tokens")

# Test parser
parser = Parser(tokens)
try:
    ast = parser.parse()
    print("\n=== PARSING SUCCESS ===")
    print(f"Parsed {len(ast.stmts)} statements:")
    for i, stmt in enumerate(ast.stmts):
        print(f"{i + 1}: {type(stmt).__name__}")
except Exception as e:
    print(f"\n=== PARSING ERROR ===")
    print(f"Error: {e}")